In [1]:
import pandas as pd
import os
from os import listdir
from scipy.signal import butter, lfilter
import librosa as lb
import soundfile as sf
import numpy as np

In [2]:
data_pd = pd.read_csv("respiratory_sound_database/Respiratory_Sound_Database/patient_diagnosis.csv", names=['id','diagnosis'])
data_pd.head()

,id,diagnosis
0,101,URTI
1,102,Healthy
2,103,Asthma
3,104,COPD
4,105,URTI


In [3]:
path = "respiratory_sound_database/Respiratory_Sound_Database/audio_and_txt_files/"
files = [f for f in os.listdir(path) if f.endswith(".wav")]

In [4]:
lowcut = 50.0
highcut = 2500.0
order = 6

In [7]:
os.mkdir("data_coding3/new")
os.mkdir("data_coding3/new/hasil")

In [8]:
loc ="data_coding3/new/hasil/"
for f in files:
    dir_audio = path+f
    data, fs = lb.core.load(dir_audio)
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b, a, data)
    norm_y = y / np.abs(y).max()
    sf.write(loc+f, norm_y, fs)

In [9]:
files.sort()
print(files)

['101_1b1_Al_sc_Meditron.wav', '101_1b1_Pr_sc_Meditron.wav', '102_1b1_Ar_sc_Meditron.wav', '103_2b2_Ar_mc_LittC2SE.wav', '104_1b1_Al_sc_Litt3200.wav', '104_1b1_Ar_sc_Litt3200.wav', '104_1b1_Ll_sc_Litt3200.wav', '104_1b1_Lr_sc_Litt3200.wav', '104_1b1_Pl_sc_Litt3200.wav', '104_1b1_Pr_sc_Litt3200.wav', '105_1b1_Tc_sc_Meditron.wav', '106_2b1_Pl_mc_LittC2SE.wav', '106_2b1_Pr_mc_LittC2SE.wav', '107_2b3_Al_mc_AKGC417L.wav', '107_2b3_Ar_mc_AKGC417L.wav', '107_2b3_Ll_mc_AKGC417L.wav', '107_2b3_Lr_mc_AKGC417L.wav', '107_2b3_Pl_mc_AKGC417L.wav', '107_2b3_Pr_mc_AKGC417L.wav', '107_2b3_Tc_mc_AKGC417L.wav', '107_2b4_Al_mc_AKGC417L.wav', '107_2b4_Ar_mc_AKGC417L.wav', '107_2b4_Ll_mc_AKGC417L.wav', '107_2b4_Lr_mc_AKGC417L.wav', '107_2b4_Pl_mc_AKGC417L.wav', '107_2b4_Pr_mc_AKGC417L.wav', '107_2b4_Tc_mc_AKGC417L.wav', '107_2b5_Al_mc_AKGC417L.wav', '107_2b5_Ar_mc_AKGC417L.wav', '107_2b5_Ll_mc_AKGC417L.wav', '107_2b5_Lr_mc_AKGC417L.wav', '107_2b5_Pl_mc_AKGC417L.wav', '107_2b5_Pr_mc_AKGC417L.wav', '107_2b5_

In [10]:
files_ = []
for f in files:
    names = f.split(".")
    df = pd.read_csv(path + '/' + names[0] + '.txt', sep='\t', names=['start', 'end', 'crackles', 'wheezes'])
    df['filename'] = names[0]
    f_features = f.split("_")
    df['id'] = f_features[0]
    df['ac_mode'] = f_features[3]
    
    files_.append(df)
    
files_df = pd.concat(files_)
files_df.reset_index()
files_df.head()

,start,end,crackles,wheezes,filename,id,ac_mode
0,0.036,0.579,0,0,101_1b1_Al_sc_Meditron,101,sc
1,0.579,2.450,0,0,101_1b1_Al_sc_Meditron,101,sc
2,2.450,3.893,0,0,101_1b1_Al_sc_Meditron,101,sc
3,3.893,5.793,0,0,101_1b1_Al_sc_Meditron,101,sc
4,5.793,7.521,0,0,101_1b1_Al_sc_Meditron,101,sc


In [11]:
files_df['id'] = files_df['id'].astype('int64')
files_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6898 entries, 0 to 10
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   start     6898 non-null   float64
 1   end       6898 non-null   float64
 2   crackles  6898 non-null   int64  
 3   wheezes   6898 non-null   int64  
 4   filename  6898 non-null   object 
 5   id        6898 non-null   int64  
 6   ac_mode   6898 non-null   object 
dtypes: float64(2), int64(3), object(2)
memory usage: 431.1+ KB


In [12]:
files_df = pd.merge(files_df, data_pd, on='id')
files_df.head()

,start,end,crackles,wheezes,filename,id,ac_mode,diagnosis
0,0.036,0.579,0,0,101_1b1_Al_sc_Meditron,101,sc,URTI
1,0.579,2.450,0,0,101_1b1_Al_sc_Meditron,101,sc,URTI
2,2.450,3.893,0,0,101_1b1_Al_sc_Meditron,101,sc,URTI
3,3.893,5.793,0,0,101_1b1_Al_sc_Meditron,101,sc,URTI
4,5.793,7.521,0,0,101_1b1_Al_sc_Meditron,101,sc,URTI


In [13]:
def slice_data(start, end, raw_data,  sample_rate):
    max_ind = len(raw_data) 
    start_ind = min(int(start * sample_rate), max_ind)
    end_ind = min(int(end * sample_rate), max_ind)
    return raw_data[start_ind: end_ind]

In [14]:
files_df['len_per_slice'] = files_df['end'].sub(files_df['start'], axis = 0)
files_df.head(25)

,start,end,crackles,wheezes,filename,id,ac_mode,diagnosis,len_per_slice
0,0.036,0.579,0,0,101_1b1_Al_sc_Meditron,101,sc,URTI,0.543
1,0.579,2.450,0,0,101_1b1_Al_sc_Meditron,101,sc,URTI,1.871
2,2.450,3.893,0,0,101_1b1_Al_sc_Meditron,101,sc,URTI,1.443
3,3.893,5.793,0,0,101_1b1_Al_sc_Meditron,101,sc,URTI,1.900
4,5.793,7.521,0,0,101_1b1_Al_sc_Meditron,101,sc,URTI,1.728
5,7.521,9.279,0,0,101_1b1_Al_sc_Meditron,101,sc,URTI,1.758
6,9.279,11.150,0,0,101_1b1_Al_sc_Meditron,101,sc,URTI,1.871
7,11.150,13.036,0,0,101_1b1_Al_sc_Meditron,101,sc,URTI,1.886
8,13.036,14.721,0,0,101_1b1_Al_sc_Meditron,101,sc,URTI,1.685
9,14.721,16.707,0,0,101_1b1_Al_sc_Meditron,101,sc,URTI,1.986


In [15]:
os.mkdir('data_coding3/new/output')

In [23]:
diag = files_df['diagnosis'].unique()

In [25]:
for d in diag:
    path = 'data_coding3/new/output/'+d
    os.makedirs(path)

In [26]:
i = 0


for idx, row in files_df.iterrows():
    filename = row['filename']
    start = row['start']
    end = row['end']
    diag = row['diagnosis']
    
    if 6 < end - start:
      end = start + 6
    
    aud_loc = 'data_coding3/new/hasil/' + filename + '.wav'
    
    if idx != 0:
        if files_df.iloc[idx-1]['filename'] == filename:
            i=i+1
        else:
            i=0
    n_filename = filename + '_' + str(i) + '.wav'
    path = 'data_coding3/new/output/' + diag + '/' + n_filename
    
    print('processing ' + n_filename + '...')

    data, samplingrate = lb.load(aud_loc)
    sliced_data = slice_data(start=start, end=end, raw_data=data, sample_rate=samplingrate)
    
    #pad audio if < forced_max_len
    a_len = samplingrate*6
    #a_len = compute_len(samp_rate=samplingrate, time=6,acquisition_mode=row['ac_mode'])
    padded_data = lb.util.pad_center(sliced_data, a_len)

    #print(len(sliced_data) ,a_len, row['ac_mode']) 

    sf.write(file=path, data=padded_data, samplerate=samplingrate)

processing 101_1b1_Al_sc_Meditron_0.wav...
processing 101_1b1_Al_sc_Meditron_1.wav...
processing 101_1b1_Al_sc_Meditron_2.wav...
processing 101_1b1_Al_sc_Meditron_3.wav...
processing 101_1b1_Al_sc_Meditron_4.wav...
processing 101_1b1_Al_sc_Meditron_5.wav...
processing 101_1b1_Al_sc_Meditron_6.wav...
processing 101_1b1_Al_sc_Meditron_7.wav...
processing 101_1b1_Al_sc_Meditron_8.wav...
processing 101_1b1_Al_sc_Meditron_9.wav...
processing 101_1b1_Al_sc_Meditron_10.wav...
processing 101_1b1_Al_sc_Meditron_11.wav...
processing 101_1b1_Pr_sc_Meditron_0.wav...
processing 101_1b1_Pr_sc_Meditron_1.wav...
processing 101_1b1_Pr_sc_Meditron_2.wav...
processing 101_1b1_Pr_sc_Meditron_3.wav...
processing 101_1b1_Pr_sc_Meditron_4.wav...
processing 101_1b1_Pr_sc_Meditron_5.wav...
processing 101_1b1_Pr_sc_Meditron_6.wav...
processing 101_1b1_Pr_sc_Meditron_7.wav...
processing 101_1b1_Pr_sc_Meditron_8.wav...
processing 101_1b1_Pr_sc_Meditron_9.wav...
processing 101_1b1_Pr_sc_Meditron_10.wav...
processi

processing 107_2b4_Ar_mc_AKGC417L_7.wav...
processing 107_2b4_Ll_mc_AKGC417L_0.wav...
processing 107_2b4_Ll_mc_AKGC417L_1.wav...
processing 107_2b4_Ll_mc_AKGC417L_2.wav...
processing 107_2b4_Ll_mc_AKGC417L_3.wav...
processing 107_2b4_Ll_mc_AKGC417L_4.wav...
processing 107_2b4_Ll_mc_AKGC417L_5.wav...
processing 107_2b4_Ll_mc_AKGC417L_6.wav...
processing 107_2b4_Ll_mc_AKGC417L_7.wav...
processing 107_2b4_Lr_mc_AKGC417L_0.wav...
processing 107_2b4_Lr_mc_AKGC417L_1.wav...
processing 107_2b4_Lr_mc_AKGC417L_2.wav...
processing 107_2b4_Lr_mc_AKGC417L_3.wav...
processing 107_2b4_Lr_mc_AKGC417L_4.wav...
processing 107_2b4_Lr_mc_AKGC417L_5.wav...
processing 107_2b4_Lr_mc_AKGC417L_6.wav...
processing 107_2b4_Lr_mc_AKGC417L_7.wav...
processing 107_2b4_Pl_mc_AKGC417L_0.wav...
processing 107_2b4_Pl_mc_AKGC417L_1.wav...
processing 107_2b4_Pl_mc_AKGC417L_2.wav...
processing 107_2b4_Pl_mc_AKGC417L_3.wav...
processing 107_2b4_Pl_mc_AKGC417L_4.wav...
processing 107_2b4_Pl_mc_AKGC417L_5.wav...
processing 

processing 109_1b1_Lr_sc_Litt3200_1.wav...
processing 109_1b1_Lr_sc_Litt3200_2.wav...
processing 109_1b1_Lr_sc_Litt3200_3.wav...
processing 109_1b1_Lr_sc_Litt3200_4.wav...
processing 109_1b1_Lr_sc_Litt3200_5.wav...
processing 109_1b1_Lr_sc_Litt3200_6.wav...
processing 109_1b1_Lr_sc_Litt3200_7.wav...
processing 109_1b1_Pl_sc_Litt3200_0.wav...
processing 109_1b1_Pl_sc_Litt3200_1.wav...
processing 109_1b1_Pl_sc_Litt3200_2.wav...
processing 109_1b1_Pl_sc_Litt3200_3.wav...
processing 109_1b1_Pl_sc_Litt3200_4.wav...
processing 109_1b1_Pl_sc_Litt3200_5.wav...
processing 109_1b1_Pl_sc_Litt3200_6.wav...
processing 109_1b1_Pl_sc_Litt3200_7.wav...
processing 109_1b1_Pl_sc_Litt3200_8.wav...
processing 109_1b1_Pr_sc_Litt3200_0.wav...
processing 109_1b1_Pr_sc_Litt3200_1.wav...
processing 109_1b1_Pr_sc_Litt3200_2.wav...
processing 109_1b1_Pr_sc_Litt3200_3.wav...
processing 109_1b1_Pr_sc_Litt3200_4.wav...
processing 109_1b1_Pr_sc_Litt3200_5.wav...
processing 109_1b1_Pr_sc_Litt3200_6.wav...
processing 

processing 113_1b1_Pl_sc_Litt3200_1.wav...
processing 113_1b1_Pl_sc_Litt3200_2.wav...
processing 113_1b1_Pl_sc_Litt3200_3.wav...
processing 113_1b1_Pl_sc_Litt3200_4.wav...
processing 113_1b1_Pl_sc_Litt3200_5.wav...
processing 113_1b1_Pl_sc_Litt3200_6.wav...
processing 113_1b1_Pl_sc_Litt3200_7.wav...
processing 113_1b1_Pr_sc_Litt3200_0.wav...
processing 113_1b1_Pr_sc_Litt3200_1.wav...
processing 113_1b1_Pr_sc_Litt3200_2.wav...
processing 113_1b1_Pr_sc_Litt3200_3.wav...
processing 113_1b1_Pr_sc_Litt3200_4.wav...
processing 113_1b1_Pr_sc_Litt3200_5.wav...
processing 113_1b1_Pr_sc_Litt3200_6.wav...
processing 113_1b1_Pr_sc_Litt3200_7.wav...
processing 113_1b1_Pr_sc_Litt3200_8.wav...
processing 114_1b4_Al_mc_AKGC417L_0.wav...
processing 114_1b4_Al_mc_AKGC417L_1.wav...
processing 114_1b4_Al_mc_AKGC417L_2.wav...
processing 114_1b4_Al_mc_AKGC417L_3.wav...
processing 114_1b4_Al_mc_AKGC417L_4.wav...
processing 114_1b4_Al_mc_AKGC417L_5.wav...
processing 114_1b4_Al_mc_AKGC417L_6.wav...
processing 

processing 120_1b1_Lr_sc_Meditron_14.wav...
processing 120_1b1_Lr_sc_Meditron_15.wav...
processing 120_1b1_Lr_sc_Meditron_16.wav...
processing 120_1b1_Lr_sc_Meditron_17.wav...
processing 120_1b1_Lr_sc_Meditron_18.wav...
processing 120_1b1_Lr_sc_Meditron_19.wav...
processing 120_1b1_Lr_sc_Meditron_20.wav...
processing 120_1b1_Lr_sc_Meditron_21.wav...
processing 120_1b1_Lr_sc_Meditron_22.wav...
processing 120_1b1_Lr_sc_Meditron_23.wav...
processing 120_1b1_Lr_sc_Meditron_24.wav...
processing 120_1b1_Lr_sc_Meditron_25.wav...
processing 120_1b1_Lr_sc_Meditron_26.wav...
processing 120_1b1_Lr_sc_Meditron_27.wav...
processing 120_1b1_Lr_sc_Meditron_28.wav...
processing 120_1b1_Lr_sc_Meditron_29.wav...
processing 120_1b1_Lr_sc_Meditron_30.wav...
processing 120_1b1_Lr_sc_Meditron_31.wav...
processing 120_1b1_Lr_sc_Meditron_32.wav...
processing 120_1b1_Pl_sc_Meditron_0.wav...
processing 120_1b1_Pl_sc_Meditron_1.wav...
processing 120_1b1_Pl_sc_Meditron_2.wav...
processing 120_1b1_Pl_sc_Meditron_3

...

processing 206_1b1_Ar_sc_Meditron_4.wav...
processing 206_1b1_Ar_sc_Meditron_5.wav...
processing 206_1b1_Ar_sc_Meditron_6.wav...
processing 206_1b1_Ar_sc_Meditron_7.wav...
processing 206_1b1_Ar_sc_Meditron_8.wav...
processing 206_1b1_Ar_sc_Meditron_9.wav...
processing 206_1b1_Ar_sc_Meditron_10.wav...
processing 206_1b1_Lr_sc_Meditron_0.wav...
processing 206_1b1_Lr_sc_Meditron_1.wav...
processing 206_1b1_Lr_sc_Meditron_2.wav...
processing 206_1b1_Lr_sc_Meditron_3.wav...
processing 206_1b1_Lr_sc_Meditron_4.wav...
processing 206_1b1_Lr_sc_Meditron_5.wav...
processing 206_1b1_Lr_sc_Meditron_6.wav...
processing 206_1b1_Lr_sc_Meditron_7.wav...
processing 206_1b1_Pl_sc_Meditron_0.wav...
processing 206_1b1_Pl_sc_Meditron_1.wav...
processing 206_1b1_Pl_sc_Meditron_2.wav...
processing 206_1b1_Pl_sc_Meditron_3.wav...
processing 206_1b1_Pl_sc_Meditron_4.wav...
processing 206_1b1_Pl_sc_Meditron_5.wav...
processing 206_1b1_Pl_sc_Meditron_6.wav...
processing 206_1b1_Pl_sc_Meditron_7.wav...
processing

processing 210_1b1_Al_sc_Meditron_15.wav...
processing 210_1b1_Al_sc_Meditron_16.wav...
processing 210_1b1_Al_sc_Meditron_17.wav...
processing 210_1b1_Ar_sc_Meditron_0.wav...
processing 210_1b1_Ar_sc_Meditron_1.wav...
processing 210_1b1_Ar_sc_Meditron_2.wav...
processing 210_1b1_Ar_sc_Meditron_3.wav...
processing 210_1b1_Ar_sc_Meditron_4.wav...
processing 210_1b1_Ar_sc_Meditron_5.wav...
processing 210_1b1_Ar_sc_Meditron_6.wav...
processing 210_1b1_Ar_sc_Meditron_7.wav...
processing 210_1b1_Ar_sc_Meditron_8.wav...
processing 210_1b1_Ar_sc_Meditron_9.wav...
processing 210_1b1_Ar_sc_Meditron_10.wav...
processing 210_1b1_Ar_sc_Meditron_11.wav...
processing 210_1b1_Ar_sc_Meditron_12.wav...
processing 211_1p2_Ar_mc_AKGC417L_0.wav...
processing 211_1p2_Ar_mc_AKGC417L_1.wav...
processing 211_1p2_Ar_mc_AKGC417L_2.wav...
processing 211_1p2_Ar_mc_AKGC417L_3.wav...
processing 211_1p2_Ar_mc_AKGC417L_4.wav...
processing 211_1p2_Ar_mc_AKGC417L_5.wav...
processing 211_1p2_Ar_mc_AKGC417L_6.wav...
proce

processing 213_1p3_Pr_mc_AKGC417L_0.wav...
processing 213_1p3_Pr_mc_AKGC417L_1.wav...
processing 213_1p3_Pr_mc_AKGC417L_2.wav...
processing 213_1p3_Pr_mc_AKGC417L_3.wav...
processing 213_1p3_Pr_mc_AKGC417L_4.wav...
processing 213_1p3_Pr_mc_AKGC417L_5.wav...
processing 213_1p3_Pr_mc_AKGC417L_6.wav...
processing 213_1p5_Al_mc_AKGC417L_0.wav...
processing 213_1p5_Al_mc_AKGC417L_1.wav...
processing 213_1p5_Al_mc_AKGC417L_2.wav...
processing 213_1p5_Al_mc_AKGC417L_3.wav...
processing 213_1p5_Al_mc_AKGC417L_4.wav...
processing 213_1p5_Al_mc_AKGC417L_5.wav...
processing 213_1p5_Ar_mc_AKGC417L_0.wav...
processing 213_1p5_Ar_mc_AKGC417L_1.wav...
processing 213_1p5_Ar_mc_AKGC417L_2.wav...
processing 213_1p5_Ar_mc_AKGC417L_3.wav...
processing 213_1p5_Ar_mc_AKGC417L_4.wav...
processing 213_1p5_Ar_mc_AKGC417L_5.wav...
processing 213_1p5_Pl_mc_AKGC417L_0.wav...
processing 213_1p5_Pl_mc_AKGC417L_1.wav...
processing 213_1p5_Pl_mc_AKGC417L_2.wav...
processing 213_1p5_Pl_mc_AKGC417L_3.wav...
processing 

processing 218_1b1_Pr_sc_Meditron_2.wav...
processing 218_1b1_Pr_sc_Meditron_3.wav...
processing 218_1b1_Pr_sc_Meditron_4.wav...
processing 218_1b1_Pr_sc_Meditron_5.wav...
processing 218_1b1_Pr_sc_Meditron_6.wav...
processing 218_1b1_Pr_sc_Meditron_7.wav...
processing 218_1b1_Pr_sc_Meditron_8.wav...
processing 218_1b1_Pr_sc_Meditron_9.wav...
processing 218_1b1_Pr_sc_Meditron_10.wav...
processing 218_1b1_Pr_sc_Meditron_11.wav...
processing 218_1b1_Pr_sc_Meditron_12.wav...
processing 218_1b1_Pr_sc_Meditron_13.wav...
processing 218_1b1_Pr_sc_Meditron_14.wav...
processing 218_1b1_Pr_sc_Meditron_15.wav...
processing 218_1b1_Pr_sc_Meditron_16.wav...
processing 218_1b1_Pr_sc_Meditron_17.wav...
processing 218_1p1_Ar_sc_Litt3200_0.wav...
processing 218_1p1_Ar_sc_Litt3200_1.wav...
processing 218_1p1_Ar_sc_Litt3200_2.wav...
processing 218_1p1_Ar_sc_Litt3200_3.wav...
processing 218_1p1_Ar_sc_Litt3200_4.wav...
processing 218_1p1_Ar_sc_Litt3200_5.wav...
processing 218_1p1_Pl_sc_Litt3200_0.wav...
pro

processing 223_1b1_Al_sc_Meditron_7.wav...
processing 223_1b1_Al_sc_Meditron_8.wav...
processing 223_1b1_Al_sc_Meditron_9.wav...
processing 223_1b1_Ar_sc_Meditron_0.wav...
processing 223_1b1_Ar_sc_Meditron_1.wav...
processing 223_1b1_Ar_sc_Meditron_2.wav...
processing 223_1b1_Ar_sc_Meditron_3.wav...
processing 223_1b1_Ar_sc_Meditron_4.wav...
processing 223_1b1_Ar_sc_Meditron_5.wav...
processing 223_1b1_Ar_sc_Meditron_6.wav...
processing 223_1b1_Ar_sc_Meditron_7.wav...
processing 223_1b1_Ar_sc_Meditron_8.wav...
processing 223_1b1_Ar_sc_Meditron_9.wav...
processing 223_1b1_Ll_sc_Meditron_0.wav...
processing 223_1b1_Ll_sc_Meditron_1.wav...
processing 223_1b1_Ll_sc_Meditron_2.wav...
processing 223_1b1_Ll_sc_Meditron_3.wav...
processing 223_1b1_Ll_sc_Meditron_4.wav...
processing 223_1b1_Ll_sc_Meditron_5.wav...
processing 223_1b1_Ll_sc_Meditron_6.wav...
processing 223_1b1_Ll_sc_Meditron_7.wav...
processing 223_1b1_Ll_sc_Meditron_8.wav...
processing 223_1b1_Ll_sc_Meditron_9.wav...
processing 

In [30]:
for f in os.listdir('data_coding3/new/output/'):
    for i in os.listdir('data_coding3/new/output/'+f):
        name = 'data_coding3/new/output/'+f+'/'+i
        y,sr = lb.load(name,sr=None)
        dur = lb.get_duration(y=y,sr=sr)
        print(sr,dur)

22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0


22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0


22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0


22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0


22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0


22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0


22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
22050 6.0
